In [1]:
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import differential_evolution
from scipy.stats.qmc import Halton
import os
import json
from susy_qm import calculate_Hamiltonian

from collections import Counter
import time

In [77]:
potential = 'DW'
cutoff = 4
shots = 1024

In [78]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)

eigenvalues, eigenvectors = np.linalg.eig(H)
min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = np.asarray(eigenvectors[:, min_index])

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

min_3_ev = eigenvalues.argsort()[:3]
#min_eigenvector = np.asarray(eigenvectors[:, min_3_ev[0]])

In [79]:
eigenvalues[min_3_ev]

array([0.90655987+0.j, 0.95063353+0.j, 1.69566635+0.j])

In [80]:
#Create operator pool
operator_pool = []
phi = 0.0
for i in range(num_qubits):
    operator_pool.append(qml.RY(phi,wires=[i]))
    #operator_pool.append(qml.RZ(phi,wires=[i]))

c_pool = []

for control in range(num_qubits):
        for target in range(num_qubits):
            if control != target:
                c_pool.append(qml.CRY(phi=phi, wires=[control, target]))

operator_pool = operator_pool + c_pool

In [81]:
# For QHO 4
# eig1 = 100
# eig2 = 000
# eig3 = 100

In [82]:
def create_circuit(params, op_list, basis_state, use_trial=False, trial_op=None, swap=False):

    param_index = 0

    if swap:
        #qml.BasisState(basis_state, wires=range(num_qubits, 2*num_qubits))
        for op in op_list:
            o = type(op)
            if o == qml.CRY:
                w0 = op.wires[0] + num_qubits
                w1 = op.wires[1] + num_qubits
                o(params[param_index], wires=[w0,w1])
                param_index += 1
            else:
                wire = op.wires[0] + num_qubits
                o(params[param_index], wires=wire)
                param_index += 1
    else:
        #qml.BasisState(basis_state, wires=range(num_qubits))
        for op in op_list:
            o = type(op)
            o(params[param_index], wires=op.wires)
            param_index +=1

        if use_trial:
            to = type(trial_op)
            to(0.0, wires=trial_op.wires)


In [83]:
dev = qml.device("default.qubit", wires=num_qubits, shots=shots)
@qml.qnode(dev)
def energy_expval(params, op_list, basis_state):

    create_circuit(params, op_list, basis_state)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [84]:
swap_dev = qml.device("default.qubit", wires=2*num_qubits, shots=shots)
@qml.qnode(swap_dev)
def swap_test(basis_state, prev_op_list, prev_params, op_list, op_params, use_trial, trial_op):

    create_circuit(prev_params, prev_op_list, basis_state, swap=True)
    create_circuit(op_params, op_list, basis_state, use_trial, trial_op=trial_op)

    qml.Barrier()
    for i in range(num_qubits):
        qml.CNOT(wires=[i, i+num_qubits])    
        qml.Hadamard(wires=i)      

    prob = qml.probs(wires=range(2*num_qubits))

    return prob

In [85]:
def overlap(basis_state, prev_op_list, prev_params, op_list, op_params, use_trial, trial_op):

    probs = swap_test(basis_state, prev_op_list, prev_params, op_list, op_params, use_trial, trial_op)

    overlap = 0
    for idx, p in enumerate(probs):

        bitstring = format(idx, '0{}b'.format(2*num_qubits))

        counter_11 = 0
        for i in range(num_qubits):
            a = int(bitstring[i])
            b = int(bitstring[i+num_qubits])
            if (a == 1 and b == 1):
                counter_11 +=1

        overlap += p*(-1)**counter_11

    return overlap

In [86]:
def multi_swap_test(basis_state, prev_op_list, prev_params, op_list, op_params, use_trial, trial_op=None, num_swap_tests=20):

    results = []
    for _ in range(num_swap_tests):

        ol = overlap(basis_state, prev_op_list, prev_params, op_list, op_params, use_trial, trial_op)
        results.append(ol)
    
    avg_ol = sum(results) / num_swap_tests

    return avg_ol

In [87]:
def loss_f(params, op_list, prev_op_list, prev_params, basis_state, beta=2.0):

    energy = energy_expval(params, op_list, basis_state)
    penalty = 0

    if len(prev_op_list) != 0:
        for prev_op, prev_param in zip(prev_op_list, prev_params):
                    ol = multi_swap_test(basis_state, prev_op, prev_param, op_list, params, use_trial=False)
                    penalty += (beta*ol)

    return energy + (penalty)

In [88]:
def compute_grad(trial_param, H, num_qubits, trial_op, op_list, op_params, basis_state):

    dev2 = qml.device("default.qubit", wires=num_qubits, shots=None)
    @qml.qnode(dev2)

    def grad_circuit(trial_param, trial_op, op_list, op_params):

        #qml.BasisState(basis_state, wires=range(num_qubits))

        param_index = 0
        for op in op_list:
            o = type(op)
            o(op_params[param_index], wires=op.wires)
            param_index +=1

        oph = type(trial_op)
        oph(trial_param, wires=trial_op.wires)

        return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))
    
    params = pnp.tensor(trial_param, requires_grad=True)
    grad_fn = qml.grad(grad_circuit)
    grad = grad_fn(params, trial_op, op_list, op_params)
    
    return grad

In [89]:
def grad_plus_overlap(basis_state, trial_op, trial_param, op_list, op_params, prev_op_list, prev_params, beta=2.0):

    grad = compute_grad(trial_param, H, num_qubits, trial_op, op_list, op_params, basis_state)
  
    penalty = 0
    if len(prev_op_list) != 0:
            for prev_op, prev_param in zip(prev_op_list, prev_params):
                ol = multi_swap_test(basis_state, prev_op, prev_param, op_list, op_params, use_trial=True, trial_op=trial_op)
                penalty += (beta*ol)
                print(f"Applying penalty {penalty} to op {trial_op} with gradient {abs(grad)}")
    
    return abs(grad) - penalty

In [90]:
# We need to generate a random seed for each process otherwise each parallelised run will have the same result
seed = (os.getpid() * int(time.time())) % 123456789

# Optimizer
num_energy_levels = 2
num_adapt_steps = 4
num_grad_checks = 20
num_vqe_runs = 1
max_iter = 250
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-2
popsize = 20

# Main ADAPT-VQE script
prev_op_list = []
prev_params = []
all_energies = []
single_ops = [qml.RY]#,qml.RZ]


basis_state = [1] + [0]*(num_qubits-1)

for e in range(num_energy_levels):

    print(f"Running for energy level: {e}")

    op_list = []
    op_params = []
    energies = []
    pool = operator_pool.copy()
    success = False

    #if e==0:
    #    basis_state = [1] + [0]*(num_qubits-1)
    #else:
    #    basis_state = [0]*(num_qubits)

    for i in range(num_adapt_steps):

        print(f"Running for adapt step: {i}")

        max_ops_list = []
        
        if i != 0:
            
            pool.remove(most_common_gate)

            if type(most_common_gate) == qml.CRY:
                cq = most_common_gate.wires[0]
                tq = most_common_gate.wires[1]

                for sop in single_ops:
                    if (sop(phi, wires=cq) not in pool):
                        pool.append(sop(phi, wires=cq))

                    if (sop(phi, wires=tq) not in pool):
                        pool.append(sop(phi, wires=tq))
        
        for trial_param in np.random.uniform(phi, phi, size=num_grad_checks):
            grad_list = []
            for trial_op in pool:
                gpo = grad_plus_overlap(basis_state, trial_op, trial_param, op_list, op_params, prev_op_list, prev_params)
                o=type(trial_op)
                grad_op = o(trial_param, wires=trial_op.wires)

                grad_list.append((grad_op,gpo))

            max_op, max_grad = max(grad_list, key=lambda x: x[1])
            max_ops_list.append(max_op)


        counter = Counter(max_ops_list)
        most_common_gate, count = counter.most_common(1)[0]
        op_list.append(most_common_gate)

        # Generate Halton sequence
        num_dimensions = len(op_list)
        num_samples = popsize
        halton_sampler = Halton(d=num_dimensions, seed=seed)
        halton_samples = halton_sampler.random(n=num_samples)
        scaled_samples = 2 * np.pi * halton_samples

        bounds = [(0, 2 * np.pi) for _ in range(num_dimensions)]
        x0 = np.concatenate((op_params, np.array([0.0])))
        
        print('Starting VQE')

        res = differential_evolution(loss_f,
                                        bounds=bounds,
                                        args=(op_list, prev_op_list, prev_params, basis_state),
                                        x0=x0,
                                        maxiter=max_iter,
                                        tol=tol,
                                        atol=abs_tol,
                                        strategy=strategy,
                                        popsize=popsize,
                                        init=scaled_samples,
                                        seed=seed
                                        )
        
        if i!=0: pre_min_e = min_e
        min_e = res.fun
        pre_op_params = op_params.copy()
        op_params = res.x

        energies.append(min_e)

        if i!=0:
            if abs(pre_min_e - min_e) < 1e-4:
                print("gradient converged")
                energies.pop()
                op_list.pop()
                final_params = pre_op_params
                success = True
                break
            if abs(min_eigenvalue-min_e) < 1e-6:
                print("Converged to min e")
                success = True
                final_params = op_params
                break

    if success == False:
        final_params = op_params

    final_ops = []
    #print(op_list)
    for op, param in zip(op_list,final_params):
        dict = {"name": op.name,
                "param": param,
                "wires": op.wires.tolist()
                }
        final_ops.append(dict)

    #print(final_ops)
    prev_op_list.append(op_list)
    prev_params.append(final_params)
    all_energies.append(energies)

Running for energy level: 0
Running for adapt step: 0
Starting VQE
Running for adapt step: 1
Starting VQE
Running for adapt step: 2
Starting VQE
Running for adapt step: 3
Starting VQE
Running for energy level: 1
Running for adapt step: 0
Applying penalty 1.271484375 to op RY(0.0, wires=[0]) with gradient 0.0
Applying penalty 1.254296875 to op RY(0.0, wires=[1]) with gradient 1.7677669529663684
Applying penalty 1.263671875 to op RY(0.0, wires=[2]) with gradient 2.474873734152916
Applying penalty 1.2640625 to op CRY(0.0, wires=[0, 1])) with gradient 0.0
Applying penalty 1.2529296875 to op CRY(0.0, wires=[0, 2])) with gradient 0.0
Applying penalty 1.26796875 to op CRY(0.0, wires=[1, 0])) with gradient 0.0
Applying penalty 1.2638671875 to op CRY(0.0, wires=[1, 2])) with gradient 0.0
Applying penalty 1.254296875 to op CRY(0.0, wires=[2, 0])) with gradient 0.0
Applying penalty 1.263671875 to op CRY(0.0, wires=[2, 1])) with gradient 0.0
Applying penalty 1.2759765625 to op RY(0.0, wires=[0]) w

In [91]:
all_energies

[[np.float64(1.012866404651064),
  np.float64(0.9848434489815998),
  np.float64(0.9820002159118566),
  np.float64(0.9506335329473413)],
 [np.float64(2.92223341560289),
  np.float64(1.7520590369117282),
  np.float64(1.6695675543342998),
  np.float64(1.6550413473734902)]]

In [92]:
eigenvalues[min_3_ev]

array([0.90655987+0.j, 0.95063353+0.j, 1.69566635+0.j])

In [29]:
prev_op_list

[[RY(np.float64(0.0), wires=[0])],
 [CRY(0.0, wires=[1, 2])),
  RY(np.float64(0.0), wires=[0]),
  RY(np.float64(0.0), wires=[2])],
 [CRY(0.0, wires=[0, 2])),
  RY(np.float64(0.0), wires=[2]),
  RY(np.float64(0.0), wires=[0])]]